In [ ]:
using LinearAlgebra
using Statistics
using IterativeSolvers
using Convex
using SCS
using COSMO
using NLsolve
using DifferentialEquations
using SparseArrays

using JLD2

using LightGraphs
using PyPlot
PyPlot.svg(true);

In [ ]:
N = 10

g = LightGraphs.grid([1, N], periodic=true)
# g = LightGraphs.erdos_renyi(N^2, 0.35)
E = incidence_matrix(g; oriented=true)

# coupling constants
K = 2.0 # coupling scale
B = (K/mean(LightGraphs.degree(g)))*ones(ne(g));

# laplacian
L = Symmetric(E*diagm(0=>B)*E')

# u, Q = eigen(L)

L_sp = dropzeros(sparse(L))

# natural frequencies are zero for twisted states
ω = zeros(nv(g))

ω = 1e-3randn(nv(g))
ω .-= mean(ω)

In [ ]:
function f_steady_state(F, δ)
    F .= ω/nv(g) - E*(B.*sin.(E'*δ))
end

# initial condition: twisted state
q = 3 # twist parameter
δ0 = 2π*q*collect(0:N-1)/N

sln = nlsolve(f_steady_state, δ0; autodiff=:forward)

@show converged(sln);

# order parameter at the fixed point
R0 = abs2(mean(exp.(1im*sln.zero)))

@show R0; 
sln.zero

In [ ]:
# check by solving ODE
function f_kuramoto(du, u, p, t)
    f_steady_state(du, u)
end

In [ ]:
# Noisy extension
δbar = sln.zero

# construct S matrix
h = complete_graph(nv(g))
Eh = incidence_matrix(h; oriented=true)
cosδbar = cos.(Eh'*δbar)
# cosδbar = ones(ne(h))

# Laplacian of the corresponding complete graph
S = Symmetric(-Eh*diagm(0=>cosδbar)*Eh'/nv(g)^2)

# weighted Laplacian expanded around fixed point
B_fp = cos.(E'*δbar).*B
L_fp = Symmetric(E*diagm(0 => B_fp)*E')

In [ ]:
eigen(S)

In [ ]:
# Eq = Convex.Variable(nv(g), nv(g))
# C = Convex.Variable(nv(g), nv(g))
Eq = Convex.Semidefinite(nv(g))
C = Convex.Semidefinite(nv(g))

# D = spdiagm(0 => u)
# St = Q'*S*Q

problem = maximize(tr(S*Eq), [
        L_fp*Eq + Eq*L_fp == C, 
        Eq*ones(N) == zeros(N),
#         sum(Eq) == 0.0,
        diag(C, 0) == 1.0,
#         tr(C) == nv(g),
        ])

In [ ]:
# Convex.solve!(problem, () -> SCS.Optimizer(verbose=true, max_iters=20000))
Convex.solve!(problem, () -> COSMO.Optimizer(verbose=true, 
        eps_abs=1e-7, eps_rel=1e-7, max_iter=150000))

In [ ]:
@show sum(C.value)
@show sum(Eq.value)
@show sum(S)

# C_constr = C.value
# E_constr = Eq.value

In [ ]:
Cs = C.value
Eqs = Eq.value

In [ ]:
eigen(Eq.value)

In [ ]:
Eq.value*ones(N)

In [ ]:
# optimal solution
problem.optval

In [ ]:
# compare to uniform noise
C_unif = (I*(1 + 1/(nv(g)-1)) - ones(nv(g), nv(g))/(nv(g) - 1))

E_uf = lyap(Array(L), -C_unif)

optval_uniform = tr(E_uf*S)

@show optval_uniform
@show optval_uniform/problem.optval

In [ ]:
# project on subspace 1^⟂
u, V = eigen(Array(L))

V0 = V[2:end,:]
L0 = V0*L*V0'
C0 = V0*C_unif*V0'
S0 = V0*S*V0'

E_uf = lyap(L0, -C0)

optval_uniform = tr(E_uf*S0)

@show optval_uniform
@show optval_uniform/problem.optval

In [ ]:
Copt = C.value
Eopt = Eq.value
optval = problem.optval

R0
jldsave("data/twisted_ring_$(N)_fix_Cii_K_$(K)_q_$(q).jld2"; g, K, B, δ0, sln, δbar, R0, Copt, Eopt, optval, optval_uniform)

In [ ]:
@show eigvals(Symmetric(C.value))
@show eigvals(Symmetric(Eq.value));

In [ ]:
fig, axs = subplots(1, 2)

ax = axs[1]
cmin, cmax = abs(minimum(C.value)), abs(maximum(C.value))
cm = maximum([cmin, cmax])
sc = ax.matshow(C.value, cmap="RdBu", vmin=-cm, vmax=cm)
fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label="noise covariance matrix C")

ax = axs[2]
cmin, cmax = abs(minimum(Eq.value)), abs(maximum(Eq.value))
cm = maximum([cmin, cmax])
sc = ax.matshow(Eq.value, cmap="RdBu", vmin=-cm, vmax=cm)
fig.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label="perturbation covariance ⟨εεᵀ⟩")

fig.tight_layout()

In [ ]:
# find a particular realization of the noise input
function G_from_C(C)
    U, Σ, V = svd(C)
    U*diagm(0 => sqrt.(Σ))*U'
end

G = G_from_C(C.value)
η = G*randn(nv(g));

In [ ]:
# visualization for grid graphs

fig, axs = subplots(1, 2, figsize=(9, 3))

ax = axs[1]
# sc = ax.matshow(reshape(η, N, N)', cmap="RdBu")
# fig.colorbar(sc, ax=ax, label="noise strength η(t)", fraction=0.046, pad=0.04)

sc = ax.matshow(reshape(C.value, N, N)', cmap="RdBu")
fig.colorbar(sc, ax=ax, label="covariance w one oscillator", fraction=0.046, pad=0.04)

ax = axs[2]
ax.plot(C.value[1,:])
ax.plot(abs.(C.value[1,:]))

fig.tight_layout()

In [ ]:
# run a SDE solution to check results

function solve_sde(σ, δ0, G; tspan=(0.0, 3000.0), dt=0.01)
    # σ is the noise strength (multiplies the noise matrix)
    function g_kuramoto(du, u, p, t)
        du .= 1/sqrt(2)*σ*G
    end
    
    prob = SDEProblem(f_kuramoto, g_kuramoto, δ0, tspan; noise_rate_prototype=zeros(size(G)...))
    sdesln = solve(prob, EM(), dt=dt)
end

function cumtrapz_avg(t::T, Y::T) where {T <: AbstractVector}
    # Estimates the cumulative time average integral 1/T ∫₀ᵀ f(t) dt using the trapezoid rule
    # where time points are in t and corresponding samples of f are in Y
    
    # Check matching vector length
    @assert length(t) == length(Y)
    
    # Initialize Output
    out = similar(t)
    out[1] = 0.0
    # Iterate over arrays
    for i in 2:length(t)
        out[i] = out[i-1] + 0.5*(t[i] - t[i-1])*(Y[i] + Y[i-1])
    end
    out[2:end] ./= (t[2:end] .- t[1])
    out[1] = out[2]

    out
end

function Rsqr_from_sde(sdesln)
    # numerically integrate and average
    Rsqrs = [abs(mean(exp.(1im*u)))^2 for u in sdesln.u]
end

In [ ]:
σ = 0.001
G = G_from_C(C.value)

sdesln = solve_sde(σ, sln.zero, G; tspan=(0.0, 1000.0))
Rsqrs_sde = Rsqr_from_sde(sdesln);
Rsqrs_avg = cumtrapz_avg(sdesln.t, Rsqrs_sde);

In [ ]:
G_unif = G_from_C(C_unif)

sdesln_unif = solve_sde(σ, sln.zero, G_unif; tspan=(0.0, 1000.0))
Rsqrs_sde_unif = Rsqr_from_sde(sdesln_unif);
Rsqrs_avg_unif = cumtrapz_avg(sdesln_unif.t, Rsqrs_sde_unif);

In [ ]:
ts = sdesln.t
ts_unif = sdesln_unif.t

jldsave("data/twisted_ring_q_$(q)_$(N)_fix_Cii_K_$(K)_sigma_$(σ)_timeseries.jld2"; Rsqrs_avg, Rsqrs_avg_unif, ts, ts_unif, σ, G)

In [ ]:
fig, ax = subplots()
ax.axhline(R0, ls=":", label="R₀²")
ax.axhline(R0 + 0.5σ^2*problem.optval, label="R₀² + 0.5σ² tr(SE)_opt", color="C1", ls=":")
ax.axhline(R0 + 0.5σ^2*optval_uniform, label="R₀² + 0.5σ² tr(SE)_unif", color="C2", ls=":")
# ax.axhline(R0 - σ^2*problem.optval)

# ax.set_ylim(R0 - 5σ^2*abs(optval_uniform), R0 + 5σ^2*abs(optval_uniform))

# ax.plot(Rsqrs_sde, alpha=0.5)
ax.plot(sdesln.t, Rsqrs_avg, color="C1", label="⟨R²⟩ optimal")
ax.plot(sdesln_unif.t, Rsqrs_avg_unif, color="C2", label="⟨R²⟩ uniform")

# ax.set_xlim(150000, 200000)

ax.legend()
ax.set_xlabel("time t")
ax.set_ylabel("order parameter")

In [ ]:
function noise_compare(σ_max, n)
    Rsqrs_opt = []
    Rsqrs_uni = []
    σs = LinRange(0.0, σ_max, n)
    
    for σ in σs
        sdesln = solve_sde(σ, sln.zero, G; tspan=(0.0, 15000.0), dt=0.01)
        Rsqrs_sde = Rsqr_from_sde(sdesln);
        Rsqrs_avg = cumtrapz_avg(sdesln.t, Rsqrs_sde);
        
        push!(Rsqrs_opt, Rsqrs_avg[end])
        
        sdesln_unif = solve_sde(σ, sln.zero, G_unif; tspan=(0.0, 15000.0), dt=0.01)
        Rsqrs_sde_unif = Rsqr_from_sde(sdesln_unif);
        Rsqrs_avg_unif = cumtrapz_avg(sdesln_unif.t, Rsqrs_sde_unif);        
        
        push!(Rsqrs_uni, Rsqrs_avg_unif[end])
    end
    
    σs, Rsqrs_opt, Rsqrs_uni
end

In [ ]:
σs, Rsqrs_opt, Rsqrs_uni = noise_compare(1.0, 21)

In [ ]:
fig, ax = subplots()

ax.plot(σs, Rsqrs_opt, label="optimal")
ax.plot(σs, Rsqrs_uni, label="uniform")
ax.plot(σs, map(σ ->  R0 + 0.5σ^2*problem.optval, σs), "--k", label="approximation")
ax.plot(σs, map(σ ->  R0 + 0.5σ^2*optval_uniform, σs), "--k", label="approximation")

# ax.set_ylim(0, 1)

ax.legend()
ax.set_xlabel("σ")
ax.set_ylabel("⟨R²⟩")

In [ ]:
# jldsave("data/periodic_line_$(N)_fix_Cii_K_$(K)_sigma_series.jld2"; R0, σs, Rsqrs_opt, Rsqrs_uni, optval, optval_uniform)